In [1]:
%run base_test.ipynb

from base import ml

In [2]:
dir_ = '../saved_models/point_multi_peakon'
dnn_factories, names = ml.dnn_factories_from_dir(dir_)
extra_facs = []
extra_names = []

# average_reg = ml.RegressorAverager(regressor_factories=dnn_factories)
# average_reg.auto_mask(gen_one_data=ml.gen_peakons_at_point, thresh=1, batch_size=100)
# average_reg_fac = ml.RegressorFactory(regressor=average_reg)

# poly_deg = 10
# poly_fac = ml.RegressorFactory(regressor=ml.PointPolyInterp(poly_deg=poly_deg))
# bilin_fac = ml.RegressorFactory(regressor=ml.PointBilinearInterp())

# extra_facs.extend([average_reg_fac, poly_fac, bilin_fac])
# extra_names.extend(['AverageReg', 'Poly{}Reg'.format(poly_deg), 'BilinearReg'])

# Warning: the polynomial interpolation takes a while.
results = ml.eval_regressors([*dnn_factories, *extra_facs], ml.gen_peakons_at_point, 100)

INFO:tensorflow:Using config: {'_model_dir': '../saved_models/point_multi_peakon/30x2_1__ScaleCenterAtPoint_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f072e4a4c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../saved_models/point_multi_peakon/30x2_1__ScaleCenterAtPoint_crelu/model.ckpt-13000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running lo

In [3]:
for result, name in sorted(zip(results, [*names, *extra_names]), 
                           key=lambda x: x[0]['loss']):
    print(name, result['loss'])

50x4_1__ScaleCenterAtPoint_crelu 0.36264114586791135
50x6_1__ScaleCenterAtPoint_crelu 0.408489521256737
30x6_1__ScaleCenterAtPoint_crelu 0.4115474101350714
30x4_1__ScaleCenterAtPoint_crelu 0.48887198807745735
400x4_1__ScaleCenterAtPoint_crelu 0.5152376601321691
40x6_1__ScaleCenterAtPoint_crelu 0.5414779101707379
600x2_1__ScaleCenterAtPoint_crelu 0.6982757630615609
40x2_1__ScaleCenterAtPoint_crelu 0.8123007405879422
600x6_1__ScaleCenterAtPoint_crelu 1.033923479729946
40x4_1__ScaleCenterAtPoint_crelu 1.1048986224280448
30x2_1__ScaleCenterAtPoint_crelu 1.2417266800545192
400x6_1__ScaleCenterAtPoint_crelu 1.319984995583182
600x4_1__ScaleCenterAtPoint_crelu 1.4450206870287865
50x2_1__ScaleCenterAtPoint_crelu 1.5563405070448904
400x2_1__ScaleCenterAtPoint_crelu 1.9696446131855279
800x6_1__ScaleCenterAtPoint_crelu 2.617447269975278
800x4_1__ScaleCenterAtPoint_crelu 3.808675081576718
800x2_1__ScaleCenterAtPoint_crelu 4.720890735670787
